# Learning Pydantic: Best Practices and Use Cases for FastAPI and LangChain

## Introduction to Pydantic

Pydantic is a Python library for **data validation and serialization** using Python type hints. It ensures that data conforms to defined schemas, making it ideal for APIs (FastAPI), AI workflows (LangChain), and structured data handling. Pydantic simplifies validation, reduces bugs, and integrates seamlessly with modern Python frameworks.

**Why Pydantic?**
- **Type Safety**: Leverages Python type hints for automatic validation.
- **Serialization**: Converts Python objects to JSON and vice versa.
- **Error Handling**: Provides detailed error messages for invalid data.
- **Integration**: Used in FastAPI for request/response validation and in LangChain for structuring AI model inputs/outputs.

**Best Practices**:
1. Use `BaseModel` for defining data schemas.
2. Leverage type annotations for fields (e.g., `str`, `int`, `List`).
3. Use validators for custom logic.
4. Enable `orm_mode` for database integration.
5. Optimize performance by precompiling models or using `model_validate_json`.

This notebook covers key Pydantic concepts, best practices, and diverse examples integrating OOP principles (Encapsulation, Abstraction, Inheritance, Polymorphism) and advanced Python concepts (decorators, iterables, memory management, etc.). Each section includes a theory explanation followed by a practical example.

**Pre-requisites**: Install Pydantic by running the following cell.

In [ ]:
!pip install pydantic
!pip install pydantic[email]  # For email validation support

## 1. Basic Pydantic Model with Encapsulation

**Concept**: Pydantic models are defined using `BaseModel`. Encapsulation (an OOP concept) is achieved by defining private attributes (using underscores) and controlling access via methods or validators.

**Use Case**: Validating user data for an API, ensuring sensitive data (e.g., email) is validated and encapsulated.

**Theory**:
- Use `BaseModel` to define a model with typed fields.
- Use `PrivateAttr` for private attributes (encapsulation).
- Use `EmailStr` for email validation.

**Flowchart**:
```mermaid
graph TD
    A[Input Data] --> B[Create Pydantic Model]
    B --> C[Validate Fields]
    C -->|Valid| D[Access Private Attributes]
    C -->|Invalid| E[Raise ValidationError]
```

**Example**: A `User` model with encapsulated email validation.

In [ ]:
from pydantic import BaseModel, EmailStr, PrivateAttr

# Define a Pydantic model with encapsulation
class User(BaseModel):
    username: str
    email: EmailStr
    _secret_token: str = PrivateAttr(default='hidden_token')  # Encapsulated private attribute

    def get_token(self) -> str:
        """Access private token (encapsulation)."""
        return self._secret_token

# Example usage
try:
    user = User(username="john_doe", email="john@example.com")
    print(f"User: {user.username}, Email: {user.email}")
    print(f"Secret Token: {user.get_token()}")  # Accessing private attribute via method
    # Invalid email example
    invalid_user = User(username="jane_doe", email="invalid_email")
except ValueError as e:
    print(f"Validation Error: {e}")

## 2. Custom Validators with Decorators

**Concept**: Pydantic allows custom validation logic using the `@field_validator` decorator. Decorators are a powerful Python feature for modifying function behavior.

**Use Case**: Validating complex data (e.g., password strength) in an API or user registration system.

**Theory**:
- Use `@field_validator` to define custom validation for specific fields.
- Combine with regular expressions (RegEx) for pattern matching.
- Decorators wrap validation logic, keeping code modular.

**Example**: A `UserRegistration` model with a password validator using RegEx, showcasing decorators.

In [ ]:
from pydantic import BaseModel, field_validator
import re

# Define a Pydantic model with custom validator
class UserRegistration(BaseModel):
    username: str
    password: str

    @field_validator('password')
    def validate_password(cls, value):
        """Ensure password is at least 8 characters with a number and special character."""
        pattern = r'^(?=.*[0-9])(?=.*[!@#$%^&*]).{8,}$'
        if not re.match(pattern, value):
            raise ValueError('Password must be 8+ characters with a number and special character')
        return value

# Example usage
try:
    user = UserRegistration(username="alice", password="Secure#123")
    print(f"Valid User: {user}")
    # Invalid password example
    invalid_user = UserRegistration(username="bob", password="weak")
except ValueError as e:
    print(f"Validation Error: {e}")

## 3. Inheritance and Polymorphism in Pydantic

**Concept**: Pydantic supports inheritance (OOP concept) and polymorphism through method overriding. This allows creating reusable base models and specialized subclasses.

**Use Case**: Modeling different types of vehicles in a fleet management API, with shared and specialized attributes.

**Theory**:
- Inherit from a base `BaseModel` to share common fields.
- Override methods to implement polymorphic behavior.
- Use `super()` to call parent class methods.

**Example**: A `Vehicle` base model with `Car` and `Truck` subclasses, demonstrating inheritance and method overriding.

In [ ]:
from pydantic import BaseModel

# Base model
class Vehicle(BaseModel):
    brand: str
    speed: float

    def description(self) -> str:
        """Base description method."""
        return f"{self.brand} vehicle with speed {self.speed} km/h"

# Subclass with polymorphism
class Car(Vehicle):
    seats: int

    def description(self) -> str:
        """Override description for Car (polymorphism)."""
        base_desc = super().description()
        return f"{base_desc}, {self.seats} seats"

# Subclass with polymorphism
class Truck(Vehicle):
    cargo_capacity: float

    def description(self) -> str:
        """Override description for Truck (polymorphism)."""
        base_desc = super().description()
        return f"{base_desc}, {self.cargo_capacity} tons capacity"

# Example usage
car = Car(brand="Toyota", speed=180.0, seats=5)
truck = Truck(brand="Volvo", speed=120.0, cargo_capacity=10.0)
print(car.description())
print(truck.description())

## 4. Abstraction with Pydantic and Iterables

**Concept**: Abstraction (OOP concept) hides implementation details, exposing only essential interfaces. Pydantic models can use abstract base classes (ABCs) and support iterables for processing collections.

**Use Case**: Managing a collection of items in an e-commerce API, with abstract methods for processing.

**Theory**:
- Use `abc.ABC` for abstract base classes.
- Implement `__iter__` to make models iterable.
- Use Pydantic for validating collection data.

**Example**: An abstract `Item` model with an iterable `Order` model.

In [ ]:
from pydantic import BaseModel
from abc import ABC, abstractmethod
from typing import List

# Abstract base class
class ItemBase(BaseModel, ABC):
    name: str
    price: float

    @abstractmethod
    def get_details(self) -> str:
        """Abstract method for item details."""
        pass

# Concrete class
class Item(ItemBase):
    def get_details(self) -> str:
        return f"{self.name}: ${self.price}"

# Iterable order model
class Order(BaseModel):
    items: List[Item]

    def __iter__(self):
        """Make Order iterable over items."""
        return iter(self.items)

# Example usage
order = Order(items=[
    Item(name="Laptop", price=999.99),
    Item(name="Mouse", price=29.99)
])
for item in order:
    print(item.get_details())

## 5. Advanced Validation with Closures and Lambda

**Concept**: Pydantic supports complex validation using closures and lambda functions. Closures allow encapsulating logic, while lambdas provide concise functional programming.

**Use Case**: Validating discount codes in an e-commerce API, with dynamic validation rules.

**Theory**:
- Use closures to create reusable validation logic.
- Use lambdas for inline validation checks.
- Combine with Pydantic’s `@field_validator`.

**Example**: A `Discount` model with a closure-based validator.

In [ ]:
from pydantic import BaseModel, field_validator
from typing import Callable

# Closure for dynamic validation
def create_code_validator(prefix: str) -> Callable[[str], str]:
    def validate_code(value: str) -> str:
        if not value.startswith(prefix):
            raise ValueError(f"Code must start with {prefix}")
        return value
    return validate_code

# Pydantic model with closure and lambda
class Discount(BaseModel):
    code: str
    amount: float

    @field_validator('code')
    def validate_discount_code(cls, value):
        """Use closure and lambda for validation."""
        validator = create_code_validator("DISC")
        # Lambda to check length
        if not (lambda x: len(x) >= 6)(value):
            raise ValueError("Code must be 6+ characters")
        return validator(value)

# Example usage
try:
    discount = Discount(code="DISC2023", amount=10.0)
    print(f"Valid Discount: {discount}")
    invalid_discount = Discount(code="SALE123", amount=5.0)
except ValueError as e:
    print(f"Validation Error: {e}")

## 6. Pydantic with FastAPI: API Integration

**Concept**: Pydantic integrates seamlessly with FastAPI for request/response validation, leveraging type hints and automatic JSON schema generation.

**Use Case**: Building a user creation endpoint in a FastAPI application.

**Theory**:
- Use Pydantic models as request/response schemas in FastAPI.
- Automatic validation and serialization reduce boilerplate code.
- FastAPI generates OpenAPI docs from Pydantic models.

**Example**: A FastAPI endpoint using a Pydantic model. (Note: Run this in a local environment with `uvicorn` for full functionality; Colab has limitations for running servers.)

In [ ]:
!pip install fastapi uvicorn

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# Pydantic model for API
class UserCreate(BaseModel):
    name: str
    age: int

@app.post("/users/")
async def create_user(user: UserCreate):
    """Create a user with validated data."""
    return {"message": f"User {user.name} created", "data": user}

# Run this locally with: uvicorn main:app --reload
# Example curl request:
# curl -X POST "http://127.0.0.1:8000/users/" -H "Content-Type: application/json" -d '{"name": "Alice", "age": 25}'
print("Run this code locally with uvicorn to test the API.")

## 7. Pydantic with LangChain: Structured AI Outputs

**Concept**: Pydantic is used in LangChain to structure and validate outputs from large language models (LLMs), ensuring consistent data formats.

**Use Case**: Structuring AI-generated responses for a chatbot.

**Theory**:
- Define Pydantic models for expected LLM output formats.
- Use with LangChain’s `with_structured_output` to enforce structure.
- Requires Pydantic V2 for modern LangChain compatibility.

**Example**: A Pydantic model for structuring chatbot responses. (Note: LangChain requires an LLM API key; this is a simplified example.)

In [ ]:
!pip install langchain langchain-openai

from pydantic import BaseModel
from langchain_openai import ChatOpenAI

# Pydantic model for structured output
class ChatResponse(BaseModel):
    response: str
    sentiment: str

# Mock LangChain setup (replace with actual LLM API key)
try:
    model = ChatOpenAI()
    structured_model = model.with_structured_output(ChatResponse)
    response = structured_model.invoke("Hello, how can I assist you?")
    print(f"Structured Response: {response}")
except Exception as e:
    print(f"Error: {e}. Set up an OpenAI API key to run this example.")

## 8. Memory Management and Optimization

**Concept**: Pydantic models can be optimized for memory usage, especially for large datasets. Python’s memory management (reference counting, garbage collection) impacts performance.

**Use Case**: Processing large datasets in an API, optimizing memory with shallow vs. deep copies.

**Theory**:
- Use `model_validate_json` for faster JSON parsing.
- Understand shallow vs. deep copies to manage mutable objects.
- Python’s reference counting increments/decrements object references; garbage collection handles cyclic references.

**Example**: Optimizing a Pydantic model for large data with deep copy.

In [ ]:
from pydantic import BaseModel
from copy import deepcopy
import sys

# Pydantic model for large dataset
class DataPoint(BaseModel):
    id: int
    values: list[float]

# Create a large dataset
data = {"id": 1, "values": [1.0, 2.0, 3.0] * 1000}

# Shallow copy (shares references to mutable objects)
shallow_copy = data.copy()
print(f"Shallow copy memory: {sys.getsizeof(shallow_copy)} bytes")

# Deep copy (independent objects)
deep_copy = deepcopy(data)
print(f"Deep copy memory: {sys.getsizeof(deep_copy)} bytes")

# Validate with Pydantic
data_point = DataPoint.model_validate_json(str(data).replace("'", '"'))
print(f"Validated DataPoint: {data_point.id}, First few values: {data_point.values[:5]}")

## 9. Testing Pydantic Models with Pytest

**Concept**: Pydantic models should be tested to ensure validation logic works. Pytest is a robust testing framework for Python.

**Use Case**: Testing a Pydantic model for an API to ensure data integrity.

**Theory**:
- Use Pytest to write unit tests for Pydantic models.
- Test valid and invalid cases to verify validation logic.
- Use `pytest.raises` to check for expected errors.

**Example**: Testing a `Product` model with Pytest.

In [ ]:
!pip install pytest

from pydantic import BaseModel, field_validator
import pytest

# Pydantic model
class Product(BaseModel):
    name: str
    price: float

    @field_validator('price')
    def price_positive(cls, value):
        if value <= 0:
            raise ValueError("Price must be positive")
        return value

# Pytest tests
def test_valid_product():
    product = Product(name="Book", price=29.99)
    assert product.name == "Book"
    assert product.price == 29.99

def test_invalid_price():
    with pytest.raises(ValueError):
        Product(name="Book", price=-10.0)

# Run tests (in Colab, use !pytest)
# Save this code to a file (e.g., test_product.py) and run: !pytest test_product.py
print("Save this code to test_product.py and run 'pytest test_product.py' locally.")

## 10. Pydantic with Generators and Recursive Functions

**Concept**: Pydantic can validate data processed by generators or recursive functions, useful for handling large datasets or hierarchical data.

**Use Case**: Validating a tree-like data structure in a LangChain application.

**Theory**:
- Generators yield data lazily, reducing memory usage.
- Recursive functions process hierarchical data.
- Pydantic validates each element in the generator or recursive structure.

**Example**: A `TreeNode` model with a generator for traversing nodes.

In [ ]:
from pydantic import BaseModel
from typing import List, Optional

# Pydantic model for tree node
class TreeNode(BaseModel):
    value: str
    children: List['TreeNode'] = []

    def traverse(self):
        """Generator to traverse tree nodes."""
        yield self
        for child in self.children:
            yield from child.traverse()

# Example usage
tree = TreeNode(
    value="Root",
    children=[
        TreeNode(value="Child1", children=[TreeNode(value="Grandchild")]),
        TreeNode(value="Child2")
    ]
)

# Traverse and validate
for node in tree.traverse():
    print(f"Node: {node.value}")

## Conclusion

This notebook covered key Pydantic concepts and best practices, with diverse examples integrating OOP principles (Encapsulation, Abstraction, Inheritance, Polymorphism) and advanced Python features (decorators, closures, iterables, generators, memory management, testing). These prepare you for using Pydantic with FastAPI (API validation) and LangChain (structured AI outputs). Experiment with these examples, modify them, and explore Pydantic’s documentation for deeper insights.

**Next Steps**:
- Explore Pydantic’s [official documentation](https://docs.pydantic.dev/).
- Build a small FastAPI project using Pydantic models.
- Experiment with LangChain’s structured output features.

Happy coding!